In [126]:

#setting
import os
os.chdir("..")
import nb_init

os.environ['CEP_ENV'] = 'local'


In [127]:
# 필요 Pyspark 모듈 import
from cep_common import spark_utils as su
from cep_common import config as con

from pyspark.sql.types import *
import pyspark.sql.functions as F

# Create SparkSession
spark = su.create_spark_session(appName="pipeline-sample")

In [128]:
file_ext = ["pdf", "png"] # 비정형 파일 리스트
src_path = "hdfs://{}/datalake/hdfs/test/**{}".format(con.HADOOP_NODE, file_ext)

In [129]:
# schema 정의
batchDataFrame = spark.read.format("binaryFile").load(src_path).limit(1)

In [130]:
#HDFS에서 이미지 읽어오기
df = spark.readStream \
    .format('binaryFile') \
    .schema(batchDataFrame.schema) \
    .load(src_path)

In [132]:
# dataframe 정리 및 컬럼 추가
df = df.select("path", "content", df.modificationTime.alias("modified_time"))
df = df.withColumn("file_type", F.element_at(F.split(F.col("path"), "\."), -1))

In [133]:
# delta table에 데이터 저장
table_name = "samples"
save_path = "hdfs://{}/datalake/delta/bronze/{}".format(con.HADOOP_NODE, table_name)

df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .trigger(processingTime='1 second') \
    .option("checkpointLocation", save_path+"/check_points") \
    .option("path", save_path) \
    .start()
    

In [135]:
# 저장된 데이터 확인
bronze_load = spark.read \
    .format("delta") \
    .load(save_path)

bronze_load.show()

+--------------------+--------------------+--------------------+---------+
|                path|             content|       modified_time|file_type|
+--------------------+--------------------+--------------------+---------+
|hdfs://tib.t3q.ai...|[25 50 44 46 2D 3...|2021-06-21 15:22:...|      pdf|
|hdfs://tib.t3q.ai...|[89 50 4E 47 0D 0...|2021-06-22 14:29:...|      png|
+--------------------+--------------------+--------------------+---------+



In [136]:
spark.stop()